## Bulk Event Loads

There are a couple of options for loading events into the backtester: load data from yahoo finance or from a data frame.

In [1]:
import os

os.chdir("..")

In [2]:
from pxtrade.events.yahoo import load_yahoo_prices

load_yahoo_prices

<function pxtrade.events.yahoo.load_yahoo_prices(instances, *args, **kwargs)>

Let's first initialise our portfolio and backtest objects. All backtests require a trade strategy: in this case we've called it DoNothing which returns no trades. The main objective here is to demonstrate the event loader.

In [3]:
from pxtrade.assets import reset, Cash, Stock, FxRate, Portfolio
from pxtrade import Trade, Strategy, Backtest, History

reset()
spy = Stock("SPY", currency_code="USD")     # SPDR S&P 500 ETF Trust (SPY) traded in USA
bhp = Stock("BHP.AX", currency_code="AUD")  # BHP Group (BHP.AX) traded in Australia
aaa = Stock("AAA", currency_code="AUD")
bbb = Stock("BBB", currency_code="AUD")

aud = Cash("AUD")
usd = Cash("USD")
audusd = FxRate("AUDUSD")
portfolio = Portfolio("USD")

class DoNothing(Strategy):
    def generate_trades(self):
        return None


backtest = Backtest(DoNothing())
backtest

### Loading events from yahoo finance
Yahoo finance will require a ticker and date range to load data. All assets will have a 'yahoo_ticker' attribute which is None by default.

In [4]:
spy.yahoo_ticker is None

True

If this yahoo_ticker attribute is None then pxtrade will use the asset code instead.

In [5]:
spy.code

'SPY'

Next we'll load adjusted close prices for spy into our backtest object.

In [6]:
from datetime import date

start_date = date(2020, 9, 1)
end_date = date(2020, 10, 1)
load_yahoo_prices(
    spy, backtest,
    start_date=start_date,
    end_date=end_date,
)

These events are now loaded into the event queue and ready to be processed when we call backtest.run.

We can use the same function to load prices for multiple assets...

In [7]:
load_yahoo_prices(
    [bhp, audusd, "^VIX"], backtest,
    start_date=start_date,
    end_date=end_date,
)

### Loading events from a data frame
Let's load random prices for each of stocks 'AAA' and 'BBB'.

In [8]:
import pandas as pd
import numpy as np

index = pd.date_range(start_date, end_date)

df = pd.DataFrame(
    index=index,
    data=np.random.randint(0, 10, size=(len(index), 2)),
    columns=["Column1", "Column2"]
)

df.head(2)

,Column1,Column2
2020-09-01,9,4
2020-09-02,9,1


In [9]:
df.tail(2)

,Column1,Column2
2020-09-30,2,8
2020-10-01,4,6


Using the load_frame_events function we can specify the object we're loading events for, the dataframe (and column) we're loading from and the event type.

In [10]:
from pxtrade.events import load_frame_events, AssetPriceEvent

load_frame_events(
    aaa,                         # load events for stock 'AAA'
    df, "Column1",               # from dataframe 'df', column 'Column1'
    backtest=backtest,           # for this backtest
    event_class=AssetPriceEvent, # and of Event type AssetPriceEvent
)

load_frame_events(
    bbb,                         # load events for stock 'BBB'
    df, "Column2",               # from dataframe 'df', column 'Column2'
    backtest=backtest,           # for this backtest
    event_class=AssetPriceEvent, # and of Event type AssetPriceEvent
)

31

### Running the backtest

We can then run our backtest and view the price history.

In [11]:
history = History(
    portfolios=portfolio,
    backtest=backtest,
)
backtest.run()

df = history.get()

In [12]:
df

,AAA,AUD,AUDUSD,BBB,BHP.AX,Portfolio,Portfolio_AAA,Portfolio_AUD,Portfolio_BBB,Portfolio_BHP.AX,Portfolio_SPY,Portfolio_USD,SPY,USD,^VIX
2020-09-01,9.0,1.0,0.737849,4.0,36.754192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,351.194183,1.0,26.120001
2020-09-02,9.0,1.0,0.733111,1.0,37.734303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,356.273865,1.0,26.570000
2020-09-03,2.0,1.0,0.725584,4.0,37.610001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,344.012939,1.0,33.599998
2020-09-04,6.0,1.0,0.725584,1.0,36.189999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,341.204163,1.0,30.750000
2020-09-05,8.0,1.0,0.725584,4.0,36.189999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,341.204163,1.0,30.750000
2020-09-06,1.0,1.0,0.728221,6.0,36.189999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,341.204163,1.0,30.750000
2020-09-07,5.0,1.0,0.728099,9.0,37.060001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,341.204163,1.0,30.750000
2020-09-08,3.0,1.0,0.720939,5.0,37.410000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,331.881470,1.0,31.459999
2020-09-09,7.0,1.0,0.727659,3.0,36.779999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,338.435272,1.0,28.809999
2020-09-10,6.0,1.0,0.726301,4.0,36.980000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,332.558777,1.0,29.709999


A few points:
*  Asset prices are displayed in local currency. The price for cash (AUD and USD in this case) is always 1.0. Stock prices and FX rates, on the other hand, are time variant.


*  'Portfolio' represents portfolio value (in base currency), though you can change this column name by explicitly changing the keyword argument 'code' when initialising portfolios. The only requirement is that portfolio codes must be unique. The 'Portfolio_' columns represent holdings in different securities. Given that our strategy doesn't trade the holdings are always zero throughout the history.


* When we loaded the prices for multiple securities we had to pass these as a list: [bhp, audusd, "^VIX"]. The last item in the list is just a string and not an asset or fx rate instance. Data for strings will be loaded into the backtest as an indicator.

In [13]:
backtest.get_indicator("^VIX")

26.700000762939453

In [14]:
aaa.price  # price events loaded from data frame earlier and processed during backtest.run

4

In [15]:
bbb.price

6

In [16]:
spy.price

337.0400085449219

In [17]:
audusd.rate

0.716650664806366

***